In [1]:
# !pip install utils
# !pip install torch
# !pip install minisom
# !pip install pyclustering
# !pip install wget 
# !pip install scikit-learn
# !pip install gower
# !pip install kmodes
# !pip install kneebow
#!pip install seaborn

In [2]:
## Libraries
import logging
import os
import numpy as np
import pandas as pd
import time
from utils import *
from utils_cascade import *

# Configurando o nível de registro (pode ser DEBUG, INFO, WARNING, ERROR, ou CRITICAL)
logging.basicConfig(level=logging.INFO)

Pegando uma base para o estudo

In [4]:
#iris_df = pd.read_parquet('docs/Dados/iris_df(teste).parquet')
#wine_df = pd.read_parquet('docs/Dados/wine_df.parquet')
pima_df = pd.read_parquet('docs/Dados/pima_df.parquet')
#boston_df = pd.read_parquet('docs/Dados/boston_df.parquet').drop(columns=['RAD'])
#abalone_df = pd.read_parquet('docs/Dados/abalone_df.parquet').drop(columns=['Sex'])
#california_df = pd.read_parquet('docs/Dados/california_df.parquet')
#gamma_df = pd.read_parquet('docs/Dados/gamma_df.parquet')
#electric_df = pd.read_parquet('docs/Dados/electric_df.parquet')
#yellow_df = pd.read_parquet('docs/Dados/yellow_df.parquet')

In [5]:
#df = iris_df
#df = wine_df
df = pima_df
#df = boston_df
#df = abalone_df
#df = california_df
#df = gamma_df
#df = electric_df
#df = yellow_df

In [4]:
print('Linhas  :', df.iloc[:,0].count())
print('Colunas :', len(df.columns))

Linhas  : 178
Colunas : 13


## ROTEIRO

In [6]:
# Ler os parâmetros do arquivo CSV
params = pd.read_csv("params_teste_full.csv", sep = ';', decimal=',').fillna(0)

# params = params.sort_values(by=['dados','mec_ausencia','pct_ausencia','cluster_algorithm','n_clusters_knn','order_cluster','order_column','dim_x_som',
#                                       'dim_y_som','sigma_som','lr_som','max_iter_som','n_clusters_kmd','n_clusters_aggcl','affinity_aggcl','metric_dbscan',
#                                       'min_samples_dbscan','eps_dbscan'], 
#                                   ascending=[True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True])

/tmp/ipykernel_54521/2718489654.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  params = pd.read_csv("params_teste_full.csv", sep = ';', decimal=',').fillna(0)


In [7]:
## Filtrando por dados
params_df = params.loc[(params['dados'] == 'pima')] 

In [9]:
#params_df

In [17]:
final_results_df = pd.DataFrame()   # Inicializar um dataFrame vazio para armazenar os resultados concatenados
resultados_cache = {}               # Inicializar um dicionário para armazenar os resultados do eraser e imputação univariada
parametros_cache = {}               # Inicializar um dicionário para armazenar os parâmetros únicos de mecha e pct_ausencia (eraser)


# Iterar sobre cada linha do arquivo CSV e executar a imputação em cascata
for _, row in params_df.iterrows():
    params = row.to_dict()  # Converter a linha do DataFrame em um dicionário de parâmetros

    # Extrair os valores de mecha e pct_ausencia dos parâmetros
    mec_ausencia = params['mec_ausencia']
    pct_ausencia = params['pct_ausencia']
    
    # Verificar se os parâmetros de mecha e pct_ausencia já foram vistos antes
    if (mec_ausencia, pct_ausencia) not in parametros_cache:
        ##############
        ### ERASER ###
        ##############    
        dados, _, _, df_incompleto = eraser_with_params(df, params)    # idx_simulacao, 

        # Salvar em parquet
        p_miss_ = str(pct_ausencia).replace('.','')+"0"    
        nome_dataset_incompleto = f'{dados}_{mec_ausencia}{p_miss_}'           
        #df_incompleto.to_parquet(f'docs/Resultado simulações/Dataset_resultados/{nome_dataset_incompleto}.parquet')
        

        ############################
        ### IMPUTAÇÃO UNIVARIADA ###
        ############################        
        df_g1_mean, df_g1_median, time_g1 = univariate_imputation_with_params(df_incompleto)

        # Salvar em parquet
        nome_dataset_media = f'{nome_dataset_incompleto}_g1_media'            
        nome_dataset_mediana = f'{nome_dataset_incompleto}_g1_mediana'     
        #df_g1_mean.to_parquet(f'docs/Resultado simulações/Dataset_resultados/{nome_dataset_media}.parquet')
        #df_g1_median.to_parquet(f'docs/Resultado simulações/Dataset_resultados/{nome_dataset_mediana}.parquet')

        # Calcular o erro      => AJEITAR!!!!! (o df tem q estar linkado com o 'base' analisada no momento)
        avg_rmse_media, avg_sim_error_media, correl_bias_media = error_metrics(df, df_g1_mean)
        avg_rmse_mediana, avg_sim_error_mediana, correl_bias_mediana = error_metrics(df, df_g1_median)
        
        # Armazenar os resultados em cache
        resultados_cache[(mec_ausencia, pct_ausencia)] = (dados, df_incompleto, df_g1_mean, df_g1_median, time_g1) #idx_simulacao, 
        parametros_cache[(mec_ausencia, pct_ausencia)] = True  # Marcar como visto


    #########################
    ### IMPUTAÇÃO CASCATA ###
    #########################    
    idx_simulacao, df_g2_imput, time_g2 = cascade_imputation_with_params(df_incompleto, params)  # idx_simulacao, 

    # Salvar o dataset restaurado por imputação cascata em parquet
    nome_dataset_cascata = f'{dados}_g2_{idx_simulacao}'            
    #df_g2_imput.to_parquet(f'docs/Resultado simulações/Dataset_resultados/{nome_dataset_cascata}.parquet')
    logging.info(f"Simulação {idx_simulacao} salvo na pasta Datasets_Imput_G2")

    # Calcular o erro
    avg_rmse_cascata, avg_sim_error_cascata, correl_bias_cascata = error_metrics(df, df_g2_imput)

    # Criar um dataframe com os resultados dos erros das imputações univariadas e cascata
    result_df = pd.DataFrame({"idx_simulacao": [idx_simulacao], 
            "avg_rmse_media": [avg_rmse_media], "avg_sim_error_media": [avg_sim_error_media], "correl_bias_media": [correl_bias_media],
            "avg_rmse_mediana": [avg_rmse_mediana], "avg_sim_error_mediana": [avg_sim_error_mediana], "correl_bias_mediana": [correl_bias_mediana], "time_g1": [time_g1],
            "avg_rmse_cascata": [avg_rmse_cascata], "avg_sim_error_cascata": [avg_sim_error_cascata], "correl_bias_cascata": [correl_bias_cascata], "time_g2": [time_g2]})
    
    final_results_df = pd.concat([final_results_df, result_df], ignore_index=True)
    final_results_df = final_results_df.drop_duplicates()

    # Salvar em um arquivo CSV
    nome_resultado = f'Resultado_simulacao_{dados}'
    #final_results_df.to_csv(f"docs/Resultado simulações/Dataset_resultados/{nome_resultado}.csv", index=False)  
    final_results_df.to_parquet(f'docs/Resultado simulações/Dataset_resultados/{nome_resultado}.parquet')
    logging.info(f"Resultados salvos")
    logging.info(f"-----------------")

logging.info(f"Simulação completa!")

# Wine:  -> h ->  dias
# Pima:  -> h ->  dias
# Boston   :  -> h ->  dias

INFO:root:Iniciando simulação 130001...
INFO:root:Eraser do MAR_0.1 concluído. Tempo decorrido: 0m.0s
INFO:root:Inicio da imputação univariada.
c:\Users\DELL\imput-cascate\notebooks\utils_cascade.py:1156: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_g1_median[name].fillna(median, inplace = True)
INFO:root:Imputação univariada concluída! Tempo decorrido: 0m.0s
INFO:root:----------------
INFO:root:Inicio da imputação cascata da simulação 130001...
INFO:root:Iniciando a morfologia da ausência usando agglomerative_cluster
INFO